In [ ]:
import pathlib
import sys

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
root_dir = "../.."

In [ ]:
root_dir_path = pathlib.Path(root_dir)
data_dir_path = root_dir_path / "data"
raw_dir_path = data_dir_path / "raw"
calendar_path = raw_dir_path / "calendar.csv"
sales_train_validation_path = raw_dir_path / "sales_train_validation.csv"
sell_prices_path = raw_dir_path / "sell_prices.csv"
interim_dir_path = data_dir_path / "interim"
interim_path = interim_dir_path / "interim.parquet"
src_dir_path = root_dir_path / "src"

In [ ]:
sys.path.append(str(src_dir_path))

In [ ]:
from package.constants import *
from package.feature_extraction import *
from package.preprocessing import *
from package.utils import *

## calendar

In [ ]:
calendar = pd.read_csv(calendar_path, dtype=dtype, parse_dates=parse_dates)

In [ ]:
calendar.drop(columns="wday", inplace=True)

In [ ]:
create_calendar_features(calendar, "date")
create_event_name(calendar)
create_event_type(calendar)

In [ ]:
reduce_memory_usage(calendar)

## sales_train_validation

In [ ]:
sales_train_validation = pd.read_csv(sales_train_validation_path, dtype=dtype)

In [ ]:
for i in range(train_days + 1, train_days + 1 + 2 * test_days):
    sales_train_validation[f"d_{i}"] = np.nan

In [ ]:
reduce_memory_usage(sales_train_validation)

## sell_prices

In [ ]:
sell_prices = pd.read_csv(sell_prices_path, dtype=dtype)

In [ ]:
create_aggregated_features(sell_prices, ["sell_price"])
create_pct_change_features(sell_prices, ["sell_price"], periods)

In [ ]:
reduce_memory_usage(sell_prices)

## interim

In [ ]:
interim = sales_train_validation.melt(
    id_vars=["id", "item_id", "store_id", "dept_id", "cat_id", "state_id"],
    var_name="d",
    value_name=target,
)

In [ ]:
del sales_train_validation

In [ ]:
interim = interim.merge(calendar, copy=False, how="left", on="d")

In [ ]:
del calendar

In [ ]:
interim = interim.merge(
    sell_prices, copy=False, how="left", on=["store_id", "item_id", "wm_yr_wk"]
)

In [ ]:
del sell_prices

In [ ]:
interim.reset_index(drop=True, inplace=True)

In [ ]:
interim.drop(columns="wm_yr_wk", inplace=True)

In [ ]:
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/144842
create_is_holiday(interim)
# See https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/150955
create_snap(interim)

In [ ]:
# Encode categorical features
for col in ["store_id", "item_id", "dept_id", "cat_id", "state_id"]:
    interim[col] = label_encode(interim[col])

In [ ]:
reduce_memory_usage(interim)

In [ ]:
interim.info()

In [ ]:
interim.to_parquet(interim_path)